# <center>Лабораторна робота №10
## <center>Прогнозування затримок вильотів

Ваше завдання - побити єдиний бенчмарк в [змаганні](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Докладних інструкцій не буде, буде тільки тезисно описано, як отримано цей бенчмарк. Звичайно, за допомогою Xgboost. Сподіваюся, на даному етапі курсу вам досить кинути півтора погляду на дані, щоб зрозуміти, що це той тип завдання, в якій затягне Xgboost. Але перевірте ще Catboost.

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [3]:
train = pd.read_csv('flight_delays_train.csv')
test = pd.read_csv('flight_delays_test.csv')

In [3]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Отже, треба за часом вильоту літака, коду авіакомпанії-перевізника, місця вильоту і прильоту і відстані між аеропортами вильоту і прильоту передбачити затримку вильоту більше 15 хвилин. В якості найпростішого бенчмарка візьмемо логістичну регресію і дві ознаки, які найпростіше взяти: `DepTime` і` Distance`. У такій моделі результат - 0.68202 на LB. 

In [8]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, 
                                                                y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [9]:
logit = LogisticRegression(random_state=17)

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

/home/anatolii/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6789733731013721

In [10]:
logit.fit(X_train, y_train)
logit_test_pred = logit.predict_proba(X_test)[:, 1]

pd.Series(logit_test_pred, 
          name='dep_delayed_15min').to_csv('logit_2feat.csv', 
                                           index_label='id', 
                                           header=True)

/home/anatolii/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Як було отримано бенчмарк в змаганні:
- Ознаки `Distance` і` DepTime` бралися без змін
- Створено ознака "маршрут" з вихідних `Origin` і` Dest`
- До ознак `Month`,` DayofMonth`, `DayOfWeek`,` UniqueCarrier` і "маршрут" застосовано OHE-перетворення ( `LabelBinarizer`)
- Виділена відкладена вибірка
- Навчалася логістична регресія і градієнтний бустінг (xgboost), гіперпараметри бустінга налаштовувалися на крос-валідації, спочатку ті, що відповідають за складність моделі, потім число дерев фіксувалося рівним 500 і налаштовувався крок градієнтного спуску
- За допомогою `cross_val_predict` робилися прогнози обох моделей на крос-валідації (саме передбачені ймовірності), налаштовувалася лінійна суміш відповідей логістичної регресії і градиентного бустінга виду $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, де $ p_{logit}$ - передбачені логістичної регресією ймовірності класу 1, $p_{xgb}$ - аналогічно. Вага $w_1$ підбирався вручну.
- Як відповідь для тестової вибірки бралася аналогічна комбінація відповідей двох моделей, але вже навчених на всій навчальній вибірці.

Описаний план ні до чого не зобов'язує - це просто те, як рішення отримав автор завдання. Можливо, ми не захочете слідувати наміченим планом, а додасте, скажімо, пару хороших ознак і навчіть ліс з тисячі дерев.

In [3]:
import xgboost as xgb

/home/anatolii/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/anatolii/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [9]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [125]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,9,17,2,615,YV,MRY,PHX,598
1,6,8,1,739,WN,LAS,HOU,1235
2,3,11,6,651,MQ,GSP,ORD,577
3,5,17,6,1614,WN,BWI,MHT,377
4,8,27,2,1505,UA,ORD,STL,258


In [17]:
X = train.drop(['dep_delayed_15min', 'Origin', 'Dest'], axis = 1)
y = train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
categorical_feature_mask = X_train.dtypes==object
categorical_cols = X_train.columns[categorical_feature_mask].tolist()
for feature in categorical_cols:
    X_train[feature] = le.fit_transform(X_train[feature])
    X_test[feature] = le.transform(X_test[feature])

/home/anatolii/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/anatolii/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [7]:
import xgboost as xgb
from sklearn import metrics
def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

model = xgb.XGBClassifier(max_depth=50, min_child_weight=1,  n_estimators=200,\
                          n_jobs=-1 , verbose=1,learning_rate=0.16)
model.fit(X_train,y_train)

auc(model, X_train, X_test)

[10:03:03] WARNING: /workspace/src/learner.cc:480: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




(1.0, 0.7017564878005611)

In [19]:
import catboost as cb
def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

clf = cb.CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 500, l2_leaf_reg= 9, learning_rate= 0.15)
clf.fit(X_train, y_train)
auc(clf, X_train, X_test)


0:	total: 98.2ms	remaining: 49s
1:	total: 136ms	remaining: 33.9s
2:	total: 174ms	remaining: 28.9s
3:	total: 210ms	remaining: 26s
4:	total: 250ms	remaining: 24.8s
5:	total: 287ms	remaining: 23.7s
6:	total: 334ms	remaining: 23.5s
7:	total: 373ms	remaining: 23s
8:	total: 412ms	remaining: 22.5s
9:	total: 454ms	remaining: 22.2s
10:	total: 494ms	remaining: 21.9s
11:	total: 535ms	remaining: 21.7s
12:	total: 582ms	remaining: 21.8s
13:	total: 627ms	remaining: 21.8s
14:	total: 667ms	remaining: 21.6s
15:	total: 704ms	remaining: 21.3s
16:	total: 744ms	remaining: 21.1s
17:	total: 787ms	remaining: 21.1s
18:	total: 825ms	remaining: 20.9s
19:	total: 861ms	remaining: 20.7s
20:	total: 899ms	remaining: 20.5s
21:	total: 939ms	remaining: 20.4s
22:	total: 987ms	remaining: 20.5s
23:	total: 1.02s	remaining: 20.3s
24:	total: 1.06s	remaining: 20.1s
25:	total: 1.1s	remaining: 20s
26:	total: 1.14s	remaining: 19.9s
27:	total: 1.18s	remaining: 20s
28:	total: 1.24s	remaining: 20.2s
29:	total: 1.3s	remaining: 20.4s
3

(0.92460629328707, 0.7149711251324056)